In [1]:
from pyquery import PyQuery as pq
import urllib2
import requests
import pandas as pd
import re
import time
import numpy as np
import json
from pprint import PrettyPrinter as pp

In [17]:
with open('./candidate_pages.json') as data_file:    
    candidate_json = json.load(data_file)
    
candidate_json.copy()

In [18]:
for key,val in candidate_json.iteritems():
    if isinstance( val, int ):
        print key
    elif len(val) < 5:
        print key

In [92]:
def get_crimdata(year, page_text, url):
    crim_list = []
    crim_det_list = []
    success = 0
    crim_dict = {'year_election': year, 'url': url}
    header = ['serial_no','IPC_sections','Other']
    for h3 in pq(page_text)("h3"):
        if h3.text == "Details of Criminal Cases":
            tablecrims = pq(h3).next('table')
            for h3_2 in pq(tablecrims)("h3"):           
                if h3_2.text == "Cases where charges framed":
                    framed = pq(h3_2).next('table')
                    crim_dict['type'] = 'Framed'
                    for i,tr in enumerate(pq(framed)('tr')):
                        crim_dict_copy = crim_dict.copy()
                        if i > 0:
                            for j,child in enumerate(tr.iterchildren()):
                                if child.text != '---------':
                                    crim_dict_copy[header[j]] = child.text
                                    success = 1
                            if success == 1:
                                crim_list.append(crim_dict_copy)
                                success = 0
                    
                if h3_2.text == "Cases where Cognizance taken":
                    framed = pq(h3_2).next('table')
                    crim_dict['type'] = 'Cogniz'
                    for i,tr in enumerate(pq(framed)('tr')):
                        crim_dict_copy = crim_dict.copy()
                        if i > 0:
                            for j,child in enumerate(tr.iterchildren()):
                                if child.text != '---------':
                                    crim_dict_copy[header[j]] = child.text
                                    success = 1
                            if success == 1:
                                crim_list.append(crim_dict_copy)
                                success = 0
                                
                    
                if h3_2.text == "Cases where convicted":
                    framed = pq(h3_2).next('table')
                    crim_dict['type'] = 'Convict'
                    for i,tr in enumerate(pq(framed)('tr')):
                        crim_dict_copy = crim_dict.copy()
                        if i > 0:
                            for j,child in enumerate(tr.iterchildren()):
                                if child.text != '---------':
                                    crim_dict_copy[header[j]] = child.text
                                    success = 1
                            if success == 1:
                                crim_list.append(crim_dict_copy)
                                success = 0

        if h3.text == "Brief Details of IPCs":
            crim_det_dict = {'url': url}
            parent = pq(h3).parent()
            details = parent.text().split(') ')
            
            for x in details:
                if len(x)>0:
                    crim_det_copy = crim_det_dict.copy()
                    crim_det_copy['detail'] = x
                    crim_det_list.append(crim_det_copy)
                
#             for x in pq(parent).children()[1:]:
#                 dets = pq(x).text()
#                 if len(dets) > 0:
#                     crim_det_copy = crim_det_dict.copy()
#                     crim_det_copy['detail'] = dets
#                     crim_det_list.append(crim_det_copy)
                
        
    return crim_det_list, crim_list



In [93]:
### FOR TESTING ###
test_index = "http://myneta.info/ls2014/candidate.php?candidate_id=3492"
page_data = candidate_json[test_index]
det_list, crim_list = get_crimdata(2014,page_data,test_index)

det_list
#crim_list

[{'detail': 'Brief Details of IPCs 2 charges related to Voluntarily causing hurt by dangerous weapons or means (IPC Section-324',
  'url': 'http://myneta.info/ls2014/candidate.php?candidate_id=3492'},
 {'detail': '2 charges related to  criminal intimidation (IPC Section-506',
  'url': 'http://myneta.info/ls2014/candidate.php?candidate_id=3492'},
 {'detail': '1 charges related to Illegal payments in connection with an election (IPC Section-171H',
  'url': 'http://myneta.info/ls2014/candidate.php?candidate_id=3492'},
 {'detail': '8 charges related to Being member of an unlawful assembly (IPC Section-143',
  'url': 'http://myneta.info/ls2014/candidate.php?candidate_id=3492'},
 {'detail': '5 charges related to  rioting (IPC Section-147',
  'url': 'http://myneta.info/ls2014/candidate.php?candidate_id=3492'},
 {'detail': '4 charges related to Every member of unlawful assembly guilty of offence committed in prosecution of common object (IPC Section-149',
  'url': 'http://myneta.info/ls2014/ca

In [102]:
crimdataset = []
crimdetailset = []
for i,urls in enumerate(candidate_json.keys()):
    page_data = candidate_json[urls]
    if (i%100) == 0:
        print ".",
    details, dataset = get_crimdata(2014,page_data,urls)
    crimdataset = crimdataset + dataset
    crimdetailset = crimdetailset + details
    
criminal_df = pd.DataFrame(crimdataset)
crim_details_df = pd.DataFrame(crimdetailset)
criminal_df.count()

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .

IPC_sections     4502
Other            4502
serial_no        4502
type             4502
url              4502
year_election    4502
dtype: int64

In [103]:
criminal_df[criminal_df.url == 'http://myneta.info/ls2014/candidate.php?candidate_id=3492'].head(5)

,IPC_sections,Other,serial_no,type,url,year_election
2204,"143, 147, 149, 323, 324, 336, 427","Judicial Magistrate Saheb, Class-1, R.T.C. 31...",1,Framed,http://myneta.info/ls2014/candidate.php?candid...,2014
2205,"143, 149, 341","Judicial Magistrate Saheb, Class-1, R.T.C. 18...",2,Framed,http://myneta.info/ls2014/candidate.php?candid...,2014
2206,"341, 143, 147","Judicial Magistrate Saheb, Class-1, R.T.C. 34...",3,Framed,http://myneta.info/ls2014/candidate.php?candid...,2014
2207,"182, 171","Judicial Magistrate Saheb, Class-1, R.T.C. 22...",4,Framed,http://myneta.info/ls2014/candidate.php?candid...,2014
2208,171H,"Judicial Magistrate Saheb, Class-1, R.T.C. 29...",5,Framed,http://myneta.info/ls2014/candidate.php?candid...,2014


In [104]:
crim_details_df[crim_details_df.url == 'http://myneta.info/ls2014/candidate.php?candidate_id=3492'].head(5)

,detail,url
7802,Brief Details of IPCs 2 charges related to Vol...,http://myneta.info/ls2014/candidate.php?candid...
7803,2 charges related to criminal intimidation (I...,http://myneta.info/ls2014/candidate.php?candid...
7804,1 charges related to Illegal payments in conne...,http://myneta.info/ls2014/candidate.php?candid...
7805,8 charges related to Being member of an unlawf...,http://myneta.info/ls2014/candidate.php?candid...
7806,5 charges related to rioting (IPC Section-147,http://myneta.info/ls2014/candidate.php?candid...


In [105]:
criminal_df.to_csv('criminal_record.csv')
crim_details_df.to_csv('criminal_details.csv')